In [ ]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming you have:
# - u: decomposed modes [4 modes × 4000 timepoints × 306 channels]
# - 200 trials × 4 tasks

# 1. Combine modes and create temporal features
def create_ltc_features(u_modes):
    """Convert MVMD outputs to LTC-ready features"""
    # u_modes shape: [num_trials, num_modes, timepoints, channels]
    features = []
    for trial in u_modes:
        # Create mode-channel-time features
        trial_features = np.stack([
            np.abs(mode)  # Using magnitude of complex components
            for mode in trial
        ], axis=-1)
        features.append(trial_features)
    return np.array(features)

# 2. Load and preprocess your data
# Assuming you've stacked all trials into a single array
all_trials = np.load('your_data.npy')  # Shape: [200 trials, 4 modes, 4000 timepoints, 306 channels]

# Create features
X = create_ltc_features(all_trials)  # Shape: [200, 4000, 306, 4]

# Create labels (assuming 4 tasks)
y = np.repeat(np.arange(4), 50)  # 50 trials per task

# 3. Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Convert to PyTorch tensors
train_data = torch.utils.data.TensorDataset(
    torch.FloatTensor(X_train).permute(0, 3, 1, 2),  # [batch, features, time, channels]
    torch.LongTensor(y_train)
)
test_data = torch.utils.data.TensorDataset(
    torch.FloatTensor(X_test).permute(0, 3, 1, 2),
    torch.LongTensor(y_test)
)

# Create data loaders
batch_size = 16
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)